## Version Control
Check and display versions for future reproduction and debugging purposes

In [21]:
import pandas as pd
import sklearn
import numpy as np

pd.show_versions()


INSTALLED VERSIONS
------------------
commit: None
python: 3.5.5.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 61 Stepping 4, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.23.1
pytest: 3.6.2
pip: 10.0.1
setuptools: 39.2.0
Cython: 0.28.3
numpy: 1.14.5
scipy: 1.1.0
pyarrow: None
xarray: None
IPython: 6.4.0
sphinx: 1.7.5
patsy: 0.5.0
dateutil: 2.7.3
pytz: 2018.4
blosc: None
bottleneck: 1.2.1
tables: 3.4.4
numexpr: 2.6.5
feather: None
matplotlib: 2.2.2
openpyxl: 2.5.4
xlrd: 1.1.0
xlwt: 1.3.0
xlsxwriter: 1.0.5
lxml: 4.2.2
bs4: 4.6.0
html5lib: 1.0.1
sqlalchemy: 1.2.8
pymysql: None
psycopg2: None
jinja2: 2.10
s3fs: None
fastparquet: None
pandas_gbq: None
pandas_datareader: None


# 1) Explore Variables

Notes from Kaggle:

- pclass = Ticket class 1 = 1st, 2 = 2nd, 3 = 3rd sex = Sex
- Age = Age in years
- sibsp = # of siblings / spouses aboard the Titanic
- parch = # of parents / children aboard the Titanic
- ticket = Ticket number
- fare = Passenger fare
- cabin = Cabin number
- embarked = Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

Features not mentioned from Kaggle:

- Name
- Sex
- Survived

In [22]:
raw_df = pd.read_csv('titanic_train.csv')
raw_df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


## pandas.DataFrame.describe
- .describe() will help us more so after we reformat features to be numerical and predict missing values
- we will use this function later again

In [23]:
raw_df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Millet, Mr. Francis Davis",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


# 2) Data Processing
Making data numerical, feature engineering, and predicting missing values

## a) Convert categorical variables into indicator variables
Many variables seem to be categorical. Simply by converting these variables into indicator variables can greatly benefit the model. It also allows for these variables to be represented in numeric form.

- (Ex: Male: 1=True 0=False, Female: 1=True 0=False)

In [24]:
gender = pd.get_dummies(raw_df['Sex'])
gender.columns = ['Female','Male'] # Clean up format to adhere to consistency while remaining descriptive
gender.head(10)

,Female,Male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
5,0,1
6,0,1
7,0,1
8,1,0
9,1,0


In [25]:
wealth_class = pd.get_dummies(raw_df['Pclass'])
wealth_class.columns = ['Upper_Class','Middle_Class','Lower_Class']
wealth_class.head(10)

,Upper_Class,Middle_Class,Lower_Class
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1
5,0,0,1
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [26]:
embarked = pd.get_dummies(raw_df['Embarked'])
embarked.columns = ['Embarked_Cherbourg','Embarked_Queenstown','Embarked_Southampton']
embarked.head(10)

,Embarked_Cherbourg,Embarked_Queenstown,Embarked_Southampton
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
6,0,0,1
7,0,0,1
8,0,0,1
9,1,0,0


# b) Feature Engineering

Features requiring no further breakdown:
- Age
- SibSp
- Parch
- Fare

Remaining Features to be broken down:
- Name
- Ticket
- Cabin

### i) Exploring Name

In [27]:
print(str(raw_df['Name'].isnull().sum())+'/'+str(raw_df['Name'].count()))
raw_df['Name'].tail(15)

0/891


876                    Gustafsson, Mr. Alfred Ossian
877                             Petroff, Mr. Nedelio
878                               Laleff, Mr. Kristo
879    Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)
880     Shelley, Mrs. William (Imanita Parrish Hall)
881                               Markun, Mr. Johann
882                     Dahlberg, Miss. Gerda Ulrika
883                    Banfield, Mr. Frederick James
884                           Sutehall, Mr. Henry Jr
885             Rice, Mrs. William (Margaret Norton)
886                            Montvila, Rev. Juozas
887                     Graham, Miss. Margaret Edith
888         Johnston, Miss. Catherine Helen "Carrie"
889                            Behr, Mr. Karl Howell
890                              Dooley, Mr. Patrick
Name: Name, dtype: object

### Observations:
- No missing variables, that's great! This variable may have some potential.
- Variable seems to follow a consistent format: (Last name, Title. First Name Middle Name)
- Last Name could be used for network analysis
- Title could be transformed into indicator variables

### Knowing that this variable follows a consistent format, let's extract last name and title
- Str.split() is a powerful built-in function, which can transform a string into array delimited by whatever is given

In [34]:
name = {'Last_Name': [], 'Title': []}

for full_name in raw_df['Name']:
    if full_name is not None:
        last_name = str(full_name).split(',')[0]
        title = str(full_name).split(',')[1].split('.')[0]
        name['Last_Name'].append(last_name)
        name['Title'].append(title)

name = pd.DataFrame(name)
name.tail(20)

,Last_Name,Title
871,Beckwith,Mrs
872,Carlsson,Mr
873,Vander Cruyssen,Mr
874,Abelson,Mrs
875,Najib,Miss
876,Gustafsson,Mr
877,Petroff,Mr
878,Laleff,Mr
879,Potter,Mrs
880,Shelley,Mrs


Title is starting to look a lot like our categorical variables previously. Let's convert this to indicator variables.
However, before we do this, let's first look at the distribution of titles before we break this down.

In [35]:
name.groupby('Title').nunique() # My favorite title is jonkheer

,Last_Name,Title
Title,,
Capt,1,1
Col,2,1
Don,1,1
Dr,7,1
Jonkheer,1,1
Lady,1,1
Major,2,1
Master,27,1
Miss,162,1


In [36]:
rare_titles = ['Lady', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'the Countess']
for title in rare_titles:
        name['Title'] = name['Title'].str.replace(title, 'Rare')
        
name['Title'] = name['Title'].str.replace('Mlle', 'Miss')
name['Title'] = name['Title'].str.replace('Ms', 'Miss')
name['Title'] = name['Title'].str.replace('Mme', 'Mrs')

title = pd.get_dummies(name['Title'])
title.tail(10)

,Master,Miss,Mr,Mrs,Rare
881,0,0,1,0,0
882,0,1,0,0,0
883,0,0,1,0,0
884,0,0,1,0,0
885,0,0,0,1,0
886,0,0,0,0,1
887,0,1,0,0,0
888,0,1,0,0,0
889,0,0,1,0,0
890,0,0,1,0,0


### ii) Exploring Cabin

In [52]:
print(str(raw_df['Cabin'].isnull().sum())+'/'+str(raw_df['PassengerId'].count()))
total = 0
for cabin in raw_df['Cabin']:
    if pd.isna(cabin):
        continue
    if len(cabin.split(' ')) > 1:
        total+=1
print(total)
raw_df['Cabin'].tail(20)

687/891
24


871            D35
872    B51 B53 B55
873            NaN
874            NaN
875            NaN
876            NaN
877            NaN
878            NaN
879            C50
880            NaN
881            NaN
882            NaN
883            NaN
884            NaN
885            NaN
886            NaN
887            B42
888            NaN
889           C148
890            NaN
Name: Cabin, dtype: object

### Observations:
- Relatively two thirds of the data for cabin is missing. Is it still worth using this variable?
- Cabin contains valuable information that can be used to predict survival such as Deck Letter and Room Number
- Deck Letter could be split into indicator variables
- Generally speaking, Room Number corresponds to the side of the ship the passenger is being housed
- Sometimes multiple cabins exist for one passenger. How do we handle that?

### Using the relatively consistent format, extract Deck Letter and Room Number

In [41]:
cabin_info = pd.DataFrame(columns = ['cabin_number','cabin_prefix'])

for cabin in raw_df['Cabin']:
    if pd.isna(cabin):
        cabin_number = 0
        cabin_prefix = ''
        cabin = pd.DataFrame({'cabin_number': [cabin_number],
                           'cabin_prefix': [cabin_prefix]})
        cabin_info = cabin_info.append(cabin, ignore_index=True)
        continue
    cabin = cabin.split(' ')[0]
    cabin_prefix = cabin[0]
    cabin_number = cabin.replace(str(cabin_prefix), '')
    cabin = pd.DataFrame({'cabin_number': [cabin_number],
                           'cabin_prefix': [cabin_prefix]})
    cabin_info = cabin_info.append(cabin, ignore_index=True)

decks = pd.get_dummies(cabin_info['cabin_prefix'])
decks.columns = ['deck_'+col for col in decks.columns]
decks.tail()
cabin_info.tail(20)

,cabin_number,cabin_prefix
871,35,D
872,51,B
873,0,
874,0,
875,0,
876,0,
877,0,
878,0,
879,50,C
880,0,


In [30]:
missing_cols = []

for col in df.columns:
    if 'Missing_' not in col:
        df['Missing_'+col] = 0

for index, row in df.iterrows():
    for col in df.columns:
        if pd.isna(row[col]):
            df.loc[index, 'Missing_'+col] = 1
            if col not in missing_cols:
                missing_cols.append(col)
            
for col in df.columns:
    if 'Missing_' not in col:
        continue
    print(col+': '+str(df[col].sum())+'/'+str(len(df)))
            
df.tail(20)
print(missing_cols)

Missing_Survived: 0/891
Missing_Pclass: 0/891
Missing_Name: 0/891
Missing_Sex: 0/891
Missing_Age: 177/891
Missing_SibSp: 0/891
Missing_Parch: 0/891
Missing_Ticket: 0/891
Missing_Fare: 0/891
Missing_Cabin: 687/891
Missing_Embarked: 2/891
['Cabin', 'Age', 'Embarked']


In [9]:
ticket_info = pd.DataFrame(columns = ['ticket_number','ticket_prefix'])

for ticket in df['Ticket']:
    if pd.isna(str(ticket)):
        ticket_number = 0
        ticket_prefix = ''
    numbers_in_ticket = [int(s) for s in ticket.split() if s.isdigit()]
    if not numbers_in_ticket:
        ticket_number = 0
    else:
        ticket_number = numbers_in_ticket[-1]
    ticket_prefix = ticket.replace(str(ticket_number), '')
    ticket = pd.DataFrame({'ticket_number': [ticket_number],
                           'ticket_prefix': [ticket_prefix]})
    ticket_info = ticket_info.append(ticket, ignore_index=True)
    
# list string replace is not working properly
ticket_info['ticket_prefix'] = ticket_info['ticket_prefix'].replace(\
    to_replace = ['A./5.','A.5.','A/5','A/5.','A/S'], value = 'A5')
ticket_info['ticket_prefix'] = ticket_info['ticket_prefix'].replace(\
    to_replace = ['A/4','A/4.','A4.'], value = 'A4')
ticket_info['ticket_prefix'] = ticket_info['ticket_prefix'].replace(\
    to_replace = ['C.A.','CA','CA.'], value = 'CA')

ticket_info['Name'] = df['Name']
ticket_info['Last_Name'] = name['last_name']
ticket_info['Cabin'] = df['Cabin']
ticket_info['Survived'] = df['Survived']
ticket_info.tail(20)

,ticket_number,ticket_prefix,Name,Last_Name,Cabin,Survived
871,11751,,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",Beckwith,D35,1
872,695,,"Carlsson, Mr. Frans Olof",Carlsson,B51 B53 B55,0
873,345765,,"Vander Cruyssen, Mr. Victor",Vander Cruyssen,NaN,0
874,3381,P/PP,"Abelson, Mrs. Samuel (Hannah Wizosky)",Abelson,NaN,1
875,2667,,"Najib, Miss. Adele Kiamie ""Jane""",Najib,NaN,1
876,7534,,"Gustafsson, Mr. Alfred Ossian",Gustafsson,NaN,0
877,349212,,"Petroff, Mr. Nedelio",Petroff,NaN,0
878,349217,,"Laleff, Mr. Kristo",Laleff,NaN,0
879,11767,,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",Potter,C50,1
880,230433,,"Shelley, Mrs. William (Imanita Parrish Hall)",Shelley,NaN,1


In [10]:
sorted_tickets = ticket_info.sort_values(by='ticket_number')
pd.crosstab(sorted_tickets['ticket_prefix'], sorted_tickets['Survived'])

Survived,0,1
ticket_prefix,,
,407,254
A./5.,2,0
A.5.,2,0
A/4,3,0
A/4.,3,0
A/5,9,1
A/5.,6,1
A/S,1,0
A4.,1,0


In [11]:
sorted_tickets = ticket_info.sort_values(by='ticket_number')
sorted_tickets.head(20)

,ticket_number,ticket_prefix,Name,Last_Name,Cabin,Survived
597,0,LINE,"Johnson, Mr. Alfred",Johnson,NaN,0
271,0,LINE,"Tornquist, Mr. William Henry",Tornquist,NaN,1
179,0,LINE,"Leonard, Mr. Lionel",Leonard,NaN,0
302,0,LINE,"Johnson, Mr. William Cahoone Jr",Johnson,NaN,0
772,3,S.O./P.P.,"Mack, Mrs. (Mary)",Mack,E77,0
841,3,S.O./P.P.,"Mudd, Mr. Thomas Charles",Mudd,NaN,0
473,541,SC/AH Basle,"Jerwan, Mrs. Amin S (Marie Marthe Thuillard)",Jerwan,D,1
545,693,,"Nicholson, Mr. Arthur Ernest",Nicholson,NaN,0
872,695,,"Carlsson, Mr. Frans Olof",Carlsson,B51 B53 B55,0
226,751,SW/PP,"Mellors, Mr. William John",Mellors,NaN,1


In [20]:
df.columns
df.loc[]

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'Missing_Survived', 'Missing_Pclass',
       'Missing_Name', 'Missing_Sex', 'Missing_Age', 'Missing_SibSp',
       'Missing_Parch', 'Missing_Ticket', 'Missing_Fare', 'Missing_Cabin',
       'Missing_Embarked'],
      dtype='object')

In [23]:
final_df =pd.concat([Title, Wealth_Class, Embarked, Gender, df[['Age','SibSp','Parch','Fare','Missing_Age','Missing_Cabin','Missing_Embarked']]], axis=1)
target = df['Survived']
final_df.tail(5)

,Master,Miss,Mr,Mrs,Rare,Upper_Class,Middle_Class,Lower_Class,Embarked_Cherbourg,Embarked_Queenstown,...,female,male,Age,SibSp,Parch,Fare,Missing_Age,Missing_Cabin,Missing_Embarked,Survived
886,0,0,0,0,1,0,1,0,0,0,...,0,1,27.0,0,0,13.00,0,1,0,0
887,0,1,0,0,0,1,0,0,0,0,...,1,0,19.0,0,0,30.00,0,0,0,1
888,0,1,0,0,0,0,0,1,0,0,...,1,0,NaN,1,2,23.45,1,1,0,0
889,0,0,1,0,0,1,0,0,1,0,...,0,1,26.0,0,0,30.00,0,0,0,1
890,0,0,1,0,0,0,0,1,0,1,...,0,1,32.0,0,0,7.75,0,1,0,0


In [ ]:
Missing_Survived: 0/891
Missing_Pclass: 0/891
Missing_Name: 0/891
Missing_Sex: 0/891
Missing_Age: 177/891
Missing_SibSp: 0/891
Missing_Parch: 0/891
Missing_Ticket: 0/891
Missing_Fare: 0/891
Missing_Cabin: 687/891
Missing_Embarked: 2/891

In [42]:
cols = []
missing_cols = []

for col in final_df.columns:
    if pd.isna(final_df[col]).any():
        if col not in missing_cols:
            missing_cols.append(col)
    if col not in missing_cols:
        cols.append(col)
        
print(cols)
print(missing_cols)

['Master', 'Miss', 'Mr', 'Mrs', 'Rare', 'Upper_Class', 'Middle_Class', 'Lower_Class', 'Embarked_Cherbourg', 'Embarked_Queenstown', 'Embarked_Southampton', 'female', 'male', 'SibSp', 'Parch', 'Fare', 'Missing_Age', 'Missing_Cabin', 'Missing_Embarked', 'Survived']
['Age']


In [43]:
from sklearn.model_selection import train_test_split

notnans = final_df.notnull().all(axis=1)
df_notnans = final_df[notnans]

# Split into 75% train and 25% test
X_train, X_test, y_train, y_test = train_test_split(df_notnans[cols], df_notnans[missing_cols],train_size=0.75,random_state=4)

C:\Users\Benjamin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=30,
                                                          random_state=0))

# Fit on the train data
regr_multirf.fit(X_train, y_train)

# Check the prediction score
score = regr_multirf.score(X_test, y_test)
print("The prediction score on the test data is {:.2f}%".format(score*100))


The prediction score on the test data is 34.92%


In [68]:
df_nans = final_df.loc[~notnans].copy()
df_nans[missing_cols] = regr_multirf.predict(df_nans[cols])
#df_nans
#final_df.loc[final_df[missing_cols].isnull(), [missing_cols]]
#final_df[missing_cols].isnull()
final_df.loc[df_nans.index] = df_nans
final_df.isnull().any().any()

False

In [69]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing.data import QuantileTransformer

distributions = [
    ('Unscaled data', final_df),
    ('Data after standard scaling',
        StandardScaler().fit_transform(final_df)),
    ('Data after min-max scaling',
        MinMaxScaler().fit_transform(final_df)),
    ('Data after max-abs scaling',
        MaxAbsScaler().fit_transform(final_df)),
    ('Data after robust scaling',
        RobustScaler(quantile_range=(25, 75)).fit_transform(final_df)),
    ('Data after quantile transformation (uniform pdf)',
        QuantileTransformer(output_distribution='uniform')
        .fit_transform(final_df)),
    ('Data after quantile transformation (gaussian pdf)',
        QuantileTransformer(output_distribution='normal')
        .fit_transform(final_df)),
    ('Data after sample-wise L2 normalizing',
        Normalizer().fit_transform(final_df))
]

In [71]:
from sklearn import svm

for distribution in distributions:
    distribution = distribution[1]
    clf = svm.SVC()
    clf.fit(X, y)
    print(distribution[1])

     Master  Miss  Mr  Mrs  Rare  Upper_Class  Middle_Class  Lower_Class  \
0         0     0   1    0     0            0             0            1   
1         0     0   0    1     0            1             0            0   
2         0     1   0    0     0            0             0            1   
3         0     0   0    1     0            1             0            0   
4         0     0   1    0     0            0             0            1   
5         0     0   1    0     0            0             0            1   
6         0     0   1    0     0            1             0            0   
7         1     0   0    0     0            0             0            1   
8         0     0   0    1     0            0             0            1   
9         0     0   0    1     0            0             1            0   
10        0     1   0    0     0            0             0            1   
11        0     1   0    0     0            1             0            0   
12        0 